# Building a Custom Chatbot with Memory: A Complete Guide using LangChain and Gemini

## Introduction
In today's AI landscape, creating chatbots that can maintain context and remember past conversations is crucial for delivering personalised, engaging user experiences. In this tutorial, we'll build a chatbot that not only responds intelligently but also remembers previous interactions, making conversations feel more natural and contextual.
We'll leverage the power of Google's Gemini model through LangChain, implement both short-term and long-term memory using vector databases, and create a robust system that can be easily extended for various applications.

## Why Memory Matters in Chatbots
Traditional chatbots treat each interaction as isolated, leading to frustrating experiences where users must repeat information. A memory-enabled chatbot can:
* Remember user preferences and personal details
* Maintain conversation context across sessions
* Provide more accurate and personalised responses
* Create a more human-like interaction experience
* Build rapport with users over time
* Learn from past interactions to improve future responses

## What You'll Learn
* Prompt Engineering: Crafting effective prompts for consistent, contextual responses
* Memory Systems: Implementing both short-term and long-term memory
* Vector Databases: Using ChromaDB for efficient semantic search
* Context Management: Maintaining conversation flow across interactions
* RAG Implementation: Building a basic Retrieval Augmented Generation system
* Error Handling: Creating robust error management and recovery mechanisms

## Prerequisites
Before we dive in, make sure you have:
* Python 3.10 or higher
* A Google Cloud account with Gemini API access
* Basic understanding of Python and APIs
* Familiarity with LangChain (optional but helpful)

In [ ]:
# # Install required packages
# !pip install langchain \
#              google.generativeai \
#              langchain-google-genai \
#              langchain-community \
#              chromadb \
#              pytest \
#              "langchain-chroma>=0.1.2"


## Project Architecture
Our chatbot architecture consists of four main components:
* __LLM Interface:__ Google's Gemini model via LangChain for natural language processing
* __Short-term Memory:__ ConversationBufferMemory for immediate context within a session
* __Long-term Memory:__ ChromaDB vector store for persistent storage across sessions
* __Memory Retrieval:__ Semantic search for finding relevant past conversations

```python
+-------------------+     +------------------+     +------------------+
|   User Input      | --> | Memory Retrieval | --> | Gemini LLM       |
+-------------------+     +------------------+     +------------------+
                                   ^                        |
                                   |                        v
                          +------------------+     +------------------+
                          | Vector Database  | <-- | Response + Memory|
                          +------------------+     +------------------+
```                         

## Step 1: Setting Up the Environment

First, let's set up our development environment and configure the necessary components:


In [3]:
import os
from typing import Dict, List, Any, Optional
import logging
from dataclasses import dataclass
from datetime import datetime
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain.memory import ConversationBufferMemory

from langchain_chroma import Chroma
from langchain.prompts import PromptTemplate
from langchain.schema import Document

import google.generativeai as genai

logger = logging.getLogger(__name__)

# Load environment variables from .env file (if available)
load_dotenv('.env')

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## Step 2: Creating a Configuration System

To make our chatbot flexible and maintainable, we'll implement a configuration system using dataclasses:

In [ ]:
@dataclass
class ChatbotConfig:
    """Configuration for the chatbot."""

    google_api_key: str
    model_name: str = 'gemini-2.0-flash'
    temperature: float = 0.7
    embedding_model: str = 'models/gemini-embedding-exp-03-07'
    chroma_persist_dir: str = './chroma_langchain_db'
    memory_k: int = 3 # Number of relevant memories to retrieve



## Step 3: Building the Memory Management System

The heart of our chatbot is its dual-memory architecture. Let's implement a sophisticated memory system:

In [ ]:
class ChatbotMemory:
    """Memory management for the chatbot."""

    def __init__(self, embeddings, vector_store: Chroma, memory_k: int = 3):
        """Initilise the memory system."""
        self.vector_store = vector_store
        self.embeddings = embeddings
        self.memory_k = memory_k
        self.short_term_memory = ConversationBufferMemory()

        # Track conversation metadata
        self.conversation_count = 0 
        self.last_interaction_time = None

    def add_to_memory(self, human_input: str, ai_response: str) -> None:
        """Add conversation to both short-term and long-term memory"""

        # Update metadata
        self.conversation_count += 1
        self.last_interaction_time = datetime.now()
        metadata = {
            "timestamp": self.last_interaction_time.isoformat(),
            "conversation_id": self.conversation_count
        }        

        # Short-term memory
        self.short_term_memory.save_context(
            {"input": human_input},
            {"output": ai_response}
        )
        
        # long-term memory
        memory_text = f"Human: {human_input}\nAi: {ai_response}"

        self.vector_store.add_texts(
            texts = [memory_text],
            metadata = [metadata]
        )

    def get_relevant_memories(self, query: str) -> str:
        """Retrieve relevant past conversations."""

        docs = self.vector_store.similarity_search(query, k=self.memory_k)

        formated_memories = []
        for doc in docs:
            metadata = doc.metadata
            timestamp = metadata.get("timestamp", "Unknown time")
            formated_memories.append(f"[{timestamp}]\n{doc.page_content}")

        return "\n\n".join(formated_memories)

    def get_conversation_history(self) -> Dict[str, Any]:
        """Get the recent conversation history."""
        return self.short_term_memory.load_memory_variables({})
    
    def clear_short_term_memory(self) -> None:
        """Clear the short-term memory."""
        self.short_term_memory.clear()
    
    def get_all_memories(self) -> List[Document]:
        """Get all memories from the vector store."""
        # This is a simplified version - in practice, you'd implement pagination
        return self.vector_store.similarity_search("", k=1000)


## Step 4: Creating the Enhanced Chatbot
Now let's build our main chatbot class that brings everything together:

In [ ]:
class EnhancedChatbot:
    """Chatbot with enhanced memory capabilities."""

    def __init__(self, config: Optional[ChatbotConfig] = None):
        """Initialise the chatbot."""
        self.config = config or ChatbotConfig.from_env()
        self._setup_components()

    @staticmethod
    def format_conversation_history(history: Dict[str, Any]) -> str:
        """Format conversation history for display."""
        return history.get('history', '')

    def _setup_components(self) -> None:
        """Set up chatbot components."""

        self.llm = ChatGoogleGenerativeAI(
            model=self.config.model_name,
            temperature=self.config.temperature,
            google_api_key=self.config.google_api_key
        )

        self.embeddings = GoogleGenerativeAIEmbeddings(
            model=self.config.embedding_model
        )

        self.vector_store = Chroma(
            collection_name="conversation_memory",
            embedding_function=self.embeddings,
            persist_directory=self.config.chroma_persist_dir
        )

        self.memory = ChatbotMemory(
            vector_store=self.vector_store,
            embeddings=self.embeddings,
            memory_k=self.config.memory_k
        )

        self.prompt_template = self._create_prompt_template()

    def _create_prompt_template(self) -> PromptTemplate:
        """Create the prompt template for the chatbot."""
        template = """You are a helpful AI assistant with memory of past converstaions.
        
        Relevant past conversations:
        {relevant_memories}

        Recent conversation:
        {recent_history}

        Human: {input}
        AI Assistant:
        """

        return PromptTemplate(
            input_variables=["relevant_memories", "recent_history", "input"],
            template=template
        )

    def generate_response(self, user_input: str) -> Dict[str, Any]:
        """Generate a response to user input."""
        try:
            relevant_memories = self.memory.get_relevant_memories(user_input)

            recent_history = self.__class__.format_conversation_history(
                self.memory.get_conversation_history()
            )

            prompt = self.prompt_template.format(
                relevant_memories=relevant_memories,
                recent_history=recent_history,
                input=user_input
            )

            # Generate response
            response = self.llm.predict(prompt)

            # Add to memory
            self.memory.add_to_memory(user_input, response)

            return {
                "response": response,
                "relevant_memories": relevant_memories,
                "success": True
            }
        
        except Exception as e:
            logger.error(f"Error generating response; {e}")
            return {
                "response": "I'm sorry, I encountered an error. Please try again.",
                "error": str(e),
                "success": False
            }
        
    def clear_memory(self) -> None:
        """Clear the chatbot's memory."""
        self.memory.clear_short_term_memory()
        logger.info("Short-term memory cleared")

    def get_conversation_history(self) -> str:
        """Get the formated conversation history."""
        return self.__class__.format_conversation_history(
            self.memory.get_conversation_history
        )


## Interactive Chat Interface

In [ ]:
def chat_interface(chatbot: EnhancedChatbot):
    """Simple interactive chat interface."""
    print("Enhanced Chatbot with Memory")
    print("Type 'exit' to quit, 'clear' to clear memory")
    print("-" * 50)
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        elif user_input.lower() == 'clear':
            chatbot.clear_memory()
            print("Memory cleared!")
            continue
        
        response = chatbot.generate_response(user_input)
        print(f"Bot: {response['response']}")
        
        if not response['success']:
            print(f"Error: {response.get('error', 'Unknown error')}")

# Uncomment to run the interactive interface
config = ChatbotConfig(google_api_key=GOOGLE_API_KEY)
chatbot = EnhancedChatbot(config)
chat_interface(chatbot)